In [3]:
import csv
import os 
print(os.getcwd())
folder = str(os.getcwd())
def get_csv_schema(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)  # Get the header row
        data_types = []
        for i, column_name in enumerate(header):
            try:
                # Attempt to convert data in the first row to int, float, etc.
                first_row = next(reader)
                _ = int(first_row[i])
                data_types.append("int")
            except ValueError:
                try:
                    _ = float(first_row[i])
                    data_types.append("float")
                except ValueError:
                     data_types.append("string")
            finally:
                file.seek(0)
                next(reader) # Reset reader to the second row for next use
        return dict(zip(header, data_types))

# Example usage:
# "C:\\Users\\scott\\Downloads\\MOCK_DATA.csv"
file = '\MOCK_DATA.csv'
file_path = folder +  file
print(file_path)
schema = get_csv_schema(file_path)
print(schema)

c:\Users\scott\OneDrive\Cloud Computing Files\data_engineering_intro\dataengineering_aws_csv
c:\Users\scott\OneDrive\Cloud Computing Files\data_engineering_intro\dataengineering_aws_csv\MOCK_DATA.csv
{'id': 'int', 'first_name': 'string', 'last_name': 'string', 'email': 'string', 'gender': 'string', 'ip_address': 'string'}


## Upload file to S3 bucket

In [4]:
pip install boto3

   ---------------------------------------- 0.0/139.9 kB ? eta -:--:--
   -------- ------------------------------ 30.7/139.9 kB 660.6 kB/s eta 0:00:01
   -------------------------------------- - 133.1/139.9 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 139.9/139.9 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.6 MB ? eta -:--:--
   - -------------------------------------- 0.4/13.6 MB 11.2 MB/s eta 0:00:02
   -- ------------------------------------- 1.0/13.6 MB 15.2 MB/s eta 0:00:01
   ------- -------------------------------- 2.7/13.6 MB 21.2 MB/s eta 0:00:01
   -------------- ------------------------- 4.9/13.6 MB 28.3 MB/s eta 0:00:01
   ---------------------- ----------------- 7.6/13.6 MB 37.4 MB/s eta 0:00:01
   ------------------------------ --------- 10.4/13.6 MB 43.5 MB/s eta 0:00:01
   -------------------------------------- - 13.1/13.6 MB 65.6 MB/s eta 0:00:01
   ---------------------------------------- 13.6/13.6 MB 50.4 MB/s eta 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.38.16 which is incompatible.


In [ ]:
import logging 
import boto3 
from botocore.exceptions import ClientError 
import os 

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket
    
    :param file_name: File to upload
    param bucket: Bucket to upload to
    param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)
        print(object_name)
    # Upload the file
    s3_client = boto3.client('s3')
    try:
        # Upload the file to s3 bucket
        response = s3_client.upload_file(file_name, bucket, object_name)
        print(bucket)
    except ClientError as e:
        logging.error(e)
        return False
    return True


In [7]:
#time to upload the file
upload_file('MOCK_DATA.csv', 'scottsmockdata')

MOCK_DATA.csv
scottsmockdata


True

In [17]:
pip install psycopg2

  Obtaining dependency information for psycopg2 from https://files.pythonhosted.org/packages/33/39/5a9a229bb5414abeb86e33b8fc8143ab0aecce5a7f698a53e31367d30caa/psycopg2-2.9.10-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 487.6 kB/s eta 0:00:03
   ---- ----------------------------------- 0.1/1.2 MB 1.2 MB/s eta 0:00:01
   --------------------- ------------------ 0.6/1.2 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pymysql 
import boto3
import pandas as pd
import io
from io import StringIO

# AWS S3 setup
s3 = boto3.client('s3')
bucket_name = 'scottsmockdata'
file_key = 'MOCK_DATA.csv'

# Get the file object
response = s3.get_object(Bucket = bucket_name, Key=file_key)
csv_content = response['Body'].read().decode('utf-8')
data = pd.read_csv(StringIO(csv_content)) 

In [15]:
data.head(5)

,id,first_name,last_name,email,gender,ip_address
0,1,Ange,Pettecrew,apettecrew0@netlog.com,Male,11.63.29.83
1,2,Amber,Mariyushkin,amariyushkin1@usgs.gov,Female,29.21.15.63
2,3,King,Belliss,kbelliss2@wunderground.com,Male,123.99.118.36
3,4,Lelah,Fearnyough,lfearnyough3@mozilla.com,Female,178.55.249.67
4,5,Berthe,Loding,bloding4@wikispaces.com,Female,218.241.0.225


In [5]:
import mysql.connector

In [18]:
mydb = mysql.connector.connect(
    host="testdb.cmxsoes6mcz5.us-east-1.rds.amazonaws.com",
    user="admin",
    password="Scampers123!",
    database="testdb"
)
mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM testtable")
myresult = mycursor.fetchall()
print(myresult)
mycursor.execute("DROP TABLE IF EXISTS mockdatatable")
mycursor.execute("CREATE TABLE mockdatatable (id int, first_name varchar(255), last_name varchar(255), email varchar(255), gender varchar(255), ip_address varchar(255))")
for _, row in data.iterrows():
    sql = "INSERT INTO mockdatatable (id, first_name, last_name, email, gender, ip_address) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['id'], row['first_name'], row['last_name'], row['email'], row['gender'], row['ip_address'])
    mycursor.execute(sql, values)

mydb.commit()

[]


In [ ]:
{'id': 'int', 'first_name': 'string', 'last_name': 'string', 'email': 'string', 'gender': 'string', 'ip_address': 'string'}